# Import Data and Preprocess Data
Import the requried package

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
from collections import Counter
from random import shuffle
from random import choice
from random import randint

The total number of line in the trainning file and total number of distinct node

In [3]:
SUM_LINE = 20000
NUM_NODE = 4867136
NEG_SAMPLE = 20000
DISTINCT_NODE_LIST =[]
STARTNODE_IN_TRAIN=[]

the negative sample should set to the num of edge in the training set

This Part trying to read in the trainning file; Loading the data and process

In [4]:
# def train_data_summary():
with open('train.txt', 'r') as file:
    train_friend_list = defaultdict(list)
    source_node_choice=[]
    c = Counter()
    for line in file.readlines():
        line=line.split()
        train_friend_list[line[0]]=line[1:]
        source_node_choice.append(line[0])
        for node in line:
            c[node]+=1
    distinct_node_list = list(c)
    DISTINCT_NODE_LIST= distinct_node_list
#     return train_friend_list


In [24]:
len(train_friend_list['3563811'])

21

# Generate Negative Example

Need to aware that all the test data is from the row begnning nod; choose node from the listed in the trainning set

In [76]:
# def neg_sampling(train_friend_list):
neg_friend_list1 = defaultdict(list)
for node in source_node_choice:
    # Should choose from the list of distinct node
    while(len(neg_friend_list1[node])<=100):
        target = choice(distinct_node_list)
        if(target not in train_friend_list[node] and target!= node):
            neg_friend_list1[node].append(target)


Source from the whole trainning data

In [72]:
neg_friend_list2 = defaultdict(list)
while(len(neg_friend_list2)<=NEG_SAMPLE):
    # Should choose from the list of distinct node
    node = choice(distinct_node_list)
    target = choice(distinct_node_list)
    if(target not in train_friend_list[node] and target!= node):
        neg_friend_list2[node].append(target)

# Feature Generation

Get pos data dict, and neg data dict?
We need to generate three form of features:
1. the number of same friends
2. How many people source knows
3. How many people knows target

In [8]:
pos_pair_list = []
for source in train_friend_list:
    for target in train_friend_list[source]:
        pair=[]
        pair.append(source)
        pair.append(target)
        pos_pair_list.append(pair)

In [25]:
c['3563811']

4

In [10]:
def get_pair_list (friend_list):
    pair_list =[]
    for source in friend_list:
        num_element=0
        if(len(friend_list[source])<=100):
            for target in friend_list[source]:
                pair=[]
                pair.append(source)
                pair.append(target)
                pair_list.append(pair)
                
        else:
            while (num_element<=100):
                pair=[]
                index =randint(0,len(friend_list[source])-1)
                pair.append(source)
                pair.append(friend_list[source][index])
                num_element+=1
                pair_list.append(pair)
        
    return pair_list

In [11]:
def feature_generate (Source, Target,train_friend_list, c):
    feature_dict = {}
    feature_dict['num_same_follower']= len(list(set(train_friend_list[Source]).intersection(train_friend_list[Target])))
    feature_dict['num_source_follow']=len(train_friend_list[Source])
    feature_dict['num_follow_target'] =c[Target]
    return feature_dict

## Give Positive Feature to csv

Generate a feature set for the positive feature

In [12]:
pos_pair_list = get_pair_list(train_friend_list)

In [13]:
len(pos_pair_list)

1552386

In [14]:
feature_set = []
index =0
for pair in pos_pair_list:
    feature_set.append(feature_generate(pair[0],pair[1],train_friend_list,c))    


In [15]:
feature_set[:10]

[{'num_same_follower': 20, 'num_source_follow': 143, 'num_follow_target': 214},
 {'num_same_follower': 0, 'num_source_follow': 143, 'num_follow_target': 660},
 {'num_same_follower': 33,
  'num_source_follow': 143,
  'num_follow_target': 2240},
 {'num_same_follower': 0, 'num_source_follow': 143, 'num_follow_target': 7},
 {'num_same_follower': 6, 'num_source_follow': 143, 'num_follow_target': 55},
 {'num_same_follower': 29, 'num_source_follow': 143, 'num_follow_target': 48},
 {'num_same_follower': 53, 'num_source_follow': 143, 'num_follow_target': 419},
 {'num_same_follower': 31, 'num_source_follow': 143, 'num_follow_target': 306},
 {'num_same_follower': 46, 'num_source_follow': 143, 'num_follow_target': 867},
 {'num_same_follower': 26, 'num_source_follow': 143, 'num_follow_target': 146}]

In [16]:

pd.DataFrame(feature_set).to_csv('positive_feature2.csv')

In [68]:
def get_neg_pair_list (friend_list):
    pair_list =[]
    for source in friend_list:
        num_element=0
        for target in friend_list[source]:
            pair=[]
            pair.append(source)
            pair.append(target)
            pair_list.append(pair)
    return pair_list

## Give the negative pair feature into csv

### Generate feature for the 1 sample of the negative data (the source all from the trainning nodes)

In [78]:
neg_pair_list1=get_neg_pair_list(neg_friend_list1)

In [80]:
feature_set2_1 = []
index =0
for pair in neg_pair_list1:
    feature_set2_1.append(feature_generate(pair[0],pair[1],train_friend_list,c))    

In [86]:
pd.DataFrame(feature_set2_1).to_csv('negative_feature2_1.csv')

***
Question: The negative sample seems all have 0 common friends, that is not good
***

### Generate feature for the 2 sample

In [87]:
neg_pair_list2=get_neg_pair_list(neg_friend_list2)
feature_set2_2 = []
index =0
for pair in neg_pair_list2:
    feature_set2_2.append(feature_generate(pair[0],pair[1],train_friend_list,c))    

***
Question: The negative sample seems all have 0 common friends and 0 followers, that is not good
***

In [90]:
pd.DataFrame(feature_set2_2).to_csv('negative_feature2_2.csv')

## Test Model

Read in the data and generate test feature

In [33]:
# def read_test():
with open('test-public.txt', 'r') as f:
    test_data=[]
    for line in f.readlines()[1:]:
        test_pair =[]
        line=line.split()
        test_pair.append(line[1])
        test_pair.append(line[2])   
        test_data.append(test_pair)
#     return test_data

In [61]:
feature_set3 = []
index =0
for pair in test_data:
    feature_set3.append(feature_generate(pair[0],pair[1],train_friend_list,c))    

In [62]:
pd.DataFrame(feature_set3).to_csv('test_feature.csv')

# Trainning the Model

In [6]:
positive_set = pd.read_csv('positive_feature2.csv')
negative_set = pd.read_csv('negative_feature2_2.csv')
test = pd.read_csv('test_feature.csv')

In [19]:
positive_set['label']=1

In [31]:
positive_set=positive_set.drop(columns="Unnamed: 0")

In [26]:
negative_set['label']=0
test.drop(columns="Unnamed: 0")

,num_same_follower,num_source_follow,num_follow_target
0,0,21,29
1,0,71,9
2,0,205,17
3,0,506,36
4,0,18,46
...,...,...,...
1995,0,53,2
1996,0,95,41
1997,0,27,2
1998,0,56,3


In [29]:
negative_set=negative_set.drop(columns="Unnamed: 0")

,num_same_follower,num_source_follow,num_follow_target,label
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,2,0
...,...,...,...,...
1870363,0,0,1,0
1870364,0,0,1,0
1870365,0,0,1,0
1870366,0,0,1,0


In [33]:
train=pd.concat([positive_set,negative_set])

In [37]:
y_train = train['label'].values

KeyError: ('num_same_follower', 'num_same_follower')

In [39]:
x_train = []
x_train.append(train.num_same_follower)
x_train.append(train.num_source_follow)
x_train.append(train.num_follow_target)

In [40]:
x_train

[0          20
 1           0
 2          33
 3           0
 4           6
            ..
 1870363     0
 1870364     0
 1870365     0
 1870366     0
 1870367     0
 Name: num_same_follower, Length: 3422754, dtype: int64,
 0          143
 1          143
 2          143
 3          143
 4          143
           ... 
 1870363      0
 1870364      0
 1870365      0
 1870366      0
 1870367      0
 Name: num_source_follow, Length: 3422754, dtype: int64,
 0           214
 1           660
 2          2240
 3             7
 4            55
            ... 
 1870363       1
 1870364       1
 1870365       1
 1870366       1
 1870367       1
 Name: num_follow_target, Length: 3422754, dtype: int64]

In [ ]:
with open('test-public.txt', 'r') as f:
    test_data=[]
    for line in f.readlines()[1:]:
        line=line.split()
        
        